#Fitting MDPs with Low-Rank Models
This notebook shows how to fit the state-action value function to their low-rank models. 

Note that you must have MATLAB on the machine to use this notebook due to the dependency on an external Robust PCA library developed in MATLAB.

###Load dependencies

In [ ]:
push!(LOAD_PATH, "../mdps")
using MDPs, MATLAB, LowRankModel

###Load mountain car problem

In [ ]:
import MountainCar
const mc = MountainCar

mdp_mc = MDP(mc.state_space(), mc.action_space(), mc.transition, mc.reward)
Qmc = readcsv("../data/qmc.csv")
pmc = Policy(Qmc, mdp_mc.A)
print("")  # suppress output

###Load inverted pendulum problem

In [ ]:
import InvertedPendulum
const ip = InvertedPendulum

mdp_ip = MDP(ip.state_space(), ip.action_space(), ip.transition, ip.reward)
Qip = readcsv("../data/qip.csv")
pip = Policy(Qip, mdp_ip.A)
print("")  # suppress output

###Solve PCP by alternating directions
We make use of the exact ALM algorithm for Robust PCA developed in MATLAB by Lin et al. (See paper.)

In [ ]:
lambda_mc = 1 / sqrt(maximum(size(Qmc)))
lambda_ip = 1 / sqrt(maximum(size(Qip)))

Lmc, Smc, Imc = mxcall(:exact_alm_rpca, 3, Qmc, lambda_mc, 1e-5)
Lip, Sip, Iip = mxcall(:exact_alm_rpca, 3, Qip, lambda_ip, 1e-5)
print("")  # suppress output

###Sparsify and represent low-rank component compactly
Rank used in `rankify` is based on Robust PCA results above. 

To get `Lxx` from `Uxx`, `sxx`, and `Vxx`, simply multiply: 

`Lxx = Uxx * diagm(sxx) * Vxx'`.

In [ ]:
Umc, smc, Vmc = rankify(Lmc, 11)
Smc = sparsify(Smc)

Uip, sip, Vip = rankify(Lip, 50)
Sip = sparsify(Sip)
print("")  # suppress output

###Generate policies based on low-rank models

In [ ]:
pmc_lrm = Policy(Umc * diagm(smc) * Vmc' + Smc, mdp_mc.A)
pip_lrm = Policy(Uip * diagm(sip) * Vip' + Sip, mdp_ip.A)
print("")  # suppress output

###Visually compare mountain car policies

In [ ]:
viz_policies(mdp_mc, pmc, pmc_lrm, mc.XMIN, mc.XMAX, mc.VMIN, mc.VMAX)

###Visually compare inverted pendulum policies

In [ ]:
viz_policies(mdp_ip, pip, pip_lrm, ip.PMIN, ip.PMAX, ip.VMIN, ip.VMAX)

###Compare mountain car simulations

In [ ]:
mc_ss, mc_as = mc.simulation(mdp_mc, pmc, [-0.5, 0.0])
mc_ss_lrm, mc_as_lrm = mc.simulation(mdp_mc, pmc_lrm, [-0.5, 0.0])
viz_trajectories(mc_ss, mc_as, mc_ss_lrm, mc_as_lrm)

###Compare inverted pendulum simulations

In [ ]:
ip_ss, ip_as = ip.simulation(mdp_ip, pip, [-0.5, 0.0])
ip_ss_lrm, ip_as_lrm = ip.simulation(mdp_ip, pip_lrm, [-0.5, 0.0])
viz_trajectories(ip_ss, ip_as, ip_ss_lrm, ip_as_lrm)